In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LSTM
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import pickle
import sys
import numpy as np
import pandas as pd

In [2]:
class VanillaDiscriminator:
    def __init__(self, max_sequence_len):
        self.max_sequence_len = max_sequence_len
    
    def build_model(self):
        txt_shape = (self.max_sequence_len, 1, 1)
        model = Sequential(name='VanillaDiscriminator')
        model.add(Flatten(input_shape=txt_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        txt = Input(shape=txt_shape)
        validity = model(txt)
        self.discriminator = Model(txt, validity)
        return self.discriminator

In [3]:
class VanillaGenerator:
    def __init__(self, max_sequence_len):
        self.max_sequence_len = max_sequence_len
    
    def build_model(self):
        noise_shape = (self.max_sequence_len, 1)
        model = Sequential(name='VanillaGenerator')
        model.add(LSTM(256, input_shape=noise_shape))
        model.add(Dropout(0.2))
        model.add(Dense(noise_shape[0], activation='tanh'))
        model.summary()
        noise = Input(shape=noise_shape)
        txt = model(noise)
        self.generator = Model(noise, txt)
        return self.generator

In [4]:
class TextGAN:
    def __init__(self, discriminator, generator, max_sequence_len):
        self.discriminator = discriminator
        self.generator = generator
        self.max_sequence_len = max_sequence_len
        
    def build_model(self):
        self.discriminator.trainable = False # it's important to freeze the discriminator when training the generator
        gan_input = Input(shape=(self.max_sequence_len, 1)) # The GAN takes noise as input 
        generator_out = self.generator(gan_input) # generates text output
        gan_output = self.discriminator(generator_out) # and validates generated text 
        self.gan =  Model(gan_input, gan_output, name='GAN')
        self.gan.summary()
        return self.gan

In [5]:
class GANTrainer:
    def __init__(self):
        pass
    
    def train(self, X_train, discriminator, generator, gan, batch_size, epochs):
        half_batch = int(batch_size/2)
        for epoch in range(epochs):
            ##########################
            # train the discriminator on half-real and half-fake data
            ##########################
            # get random half-batch real data
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            txts = X_train[idx]

            # get half-batch fake data
            noise = np.random.normal(-10, 10, (half_batch, columns, 1))
            gen_txts = generator.predict(noise)
            gen_txts = np.expand_dims(gen_txts, axis=2)
            gen_txts = np.expand_dims(gen_txts, axis=3)

            # compute discriminator losses on real and fake data and average them
            d_loss_real = discriminator.train_on_batch(txts, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_txts, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            ##########################
            # train the GAN, thereby traning the generator (on full-batch data)
            # the discriminator is not trained in the GAN because it's trainable flag is set to False 
            ##########################
            noise = np.random.normal(0, 1, (batch_size, columns, 1))
            # the generator wants discriminator to mistake texts as real. Therefore send np.ones as labels
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

In [6]:
# data loading and preprocessing
data = pickle.load(open("X.p","rb"))
X_train = data
rows, columns, channels = X_train.shape
print(rows, columns, channels)
# expand the last dimension
X_train = np.expand_dims(X_train, axis=3)
print(X_train.shape)

11667 768 1
(11667, 768, 1, 1)


In [7]:
pd.Series(data.reshape(-1)).describe()

count    8.960256e+06
mean    -9.478921e-03
std      5.208932e-01
min     -9.697466e+00
25%     -2.290757e-01
50%      1.008263e-02
75%      2.507721e-01
max      5.218259e+00
dtype: float64

In [8]:
# parameter initializations
max_sequence_len = columns
batch_size = 1024
epochs = 100
optimizer = Adam(0.0002, 0.05)

In [9]:
discriminator = VanillaDiscriminator(max_sequence_len).build_model()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Model: "VanillaDiscriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               393728    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 525,313
Trainable params: 525,313
Non-trainable params: 0
________________________________________

In [10]:
generator = VanillaGenerator(max_sequence_len).build_model()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "VanillaGenerator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 768)               197376    
Total params: 461,568
Trainable params: 461,568
Non-trainable params: 0
_________________________________________________________________


In [11]:
gan = TextGAN(discriminator, generator, max_sequence_len).build_model()
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "GAN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 768, 1)]          0         
_________________________________________________________________
model_1 (Model)              (None, 768)               461568    
_________________________________________________________________
model (Model)                (None, 1)                 525313    
Total params: 986,881
Trainable params: 461,568
Non-trainable params: 525,313
_________________________________________________________________


In [24]:
# training
GANTrainer().train(X_train, discriminator, generator, gan, batch_size, epochs)

0 [D loss: 0.617149, acc.: 90.82%] [G loss: 0.693364]
1 [D loss: 0.435834, acc.: 98.83%] [G loss: 0.686573]
2 [D loss: 0.315553, acc.: 99.51%] [G loss: 0.681070]
3 [D loss: 0.263909, acc.: 99.41%] [G loss: 0.671529]
4 [D loss: 0.206514, acc.: 99.71%] [G loss: 0.663338]
5 [D loss: 0.166372, acc.: 99.71%] [G loss: 0.645630]
6 [D loss: 0.127427, acc.: 99.71%] [G loss: 0.618740]
7 [D loss: 0.088723, acc.: 99.80%] [G loss: 0.550577]
8 [D loss: 0.054997, acc.: 99.90%] [G loss: 0.140953]
9 [D loss: 0.034194, acc.: 99.80%] [G loss: 0.003710]
10 [D loss: 0.028065, acc.: 99.80%] [G loss: 0.005713]
11 [D loss: 0.026987, acc.: 99.90%] [G loss: 0.000938]
12 [D loss: 0.021318, acc.: 99.80%] [G loss: 0.000011]
13 [D loss: 0.019714, acc.: 99.90%] [G loss: 0.001052]
14 [D loss: 0.016533, acc.: 99.80%] [G loss: 0.000008]
15 [D loss: 0.015007, acc.: 99.90%] [G loss: 0.001152]
16 [D loss: 0.016457, acc.: 99.71%] [G loss: 0.000006]
17 [D loss: 0.017857, acc.: 99.90%] [G loss: 0.000006]
18 [D loss: 0.017108

In [12]:
discriminator.predict(X_train)

array([[0.58499175],
       [0.5200966 ],
       [0.50010216],
       ...,
       [0.5227137 ],
       [0.5246207 ],
       [0.37355912]], dtype=float32)

In [26]:
discriminator.save('discrinimator_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: discrinimator_model/assets


In [13]:
discriminator = tf.keras.models.load_model('discrinimator_model')

In [14]:
# evaluating on test data

In [15]:
# data loading and preprocessing
data_test = pickle.load(open("X_test.p","rb"))
X_test = data_test
rows, columns, channels = X_test.shape
print(rows, columns, channels)
# expand the last dimension
X_test = np.expand_dims(X_test, axis=3)
print(X_test.shape)

14014 768 1
(14014, 768, 1, 1)


In [51]:
import pandas as pd
pred_df = pd.DataFrame(discriminator.predict(X_test))
pred_df

,0
0,0.999412
1,0.999844
2,0.999943
3,0.999900
4,0.999872
...,...
14009,0.999681
14010,0.999841
14011,0.999943
14012,0.999892


In [56]:
print(pred_df.shape)
pred_df.describe()

(14014, 1)


,0
count,14014.000000
mean,0.999798
std,0.001221
min,0.904467
25%,0.999798
50%,0.999891
75%,0.999941
max,0.999996


In [53]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [57]:
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(pred_df))
scaled_df.shape

(14014, 1)

In [58]:
scaled_df.describe()

,0
count,14014.000000
mean,0.997932
std,0.012781
min,0.000000
25%,0.997926
50%,0.998905
75%,0.999432
max,1.000000
